In [1]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries

In [2]:
class ScriptData:
    def __init__(self):
        self.api_key = 'J6ZBYR30XCBEQYKW'
        self.stored_df = {}
    
    def __getitem__(self, script):
        return self.stored_df[script]
    
    def __setitem__(self, script, df):
        self.stored_df[script] = df
    
    def __contains__(self, script):
        if script in self.stored_df:
            return True
        
    def fetch_intraday_data(self, script):
        self.ts = TimeSeries(self.api_key, output_format='pandas')
    
    def convert_intraday_data(self, script):
        df, meta = self.ts.get_intraday(script, interval='60min')
        df.reset_index(level=0, inplace=True)
        df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        df['volume'] = df['volume'].astype(np.int64)
        df = df[::-1]
        df.reset_index(level=0, inplace=True)
        df = df.drop(['index'], axis=1)
        self.__setitem__(script, df)
        return df

In [3]:
script_data = ScriptData()
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2022-12-14 17:00:00,95.0700,95.40,94.92,95.2600,945478
1,2022-12-14 18:00:00,95.2100,95.25,95.06,95.1600,274090
2,2022-12-14 19:00:00,95.1500,95.36,95.11,95.3500,9815
3,2022-12-14 20:00:00,95.1600,95.39,95.02,95.2800,10386
4,2022-12-15 05:00:00,93.7300,93.74,93.25,93.5200,11750
...,...,...,...,...,...,...
95,2022-12-22 16:00:00,87.1900,87.83,87.10,87.7650,6501937
96,2022-12-22 17:00:00,87.7800,88.10,87.37,87.5800,1011025
97,2022-12-22 18:00:00,87.5800,87.76,87.43,87.7005,81491
98,2022-12-22 19:00:00,87.7499,87.76,87.61,87.7400,4446


In [4]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')

,timestamp,open,high,low,close,volume
0,2022-12-14 17:00:00,143.2100,143.58,143.13,143.4200,3079105
1,2022-12-14 18:00:00,143.4100,143.43,143.21,143.3400,989150
2,2022-12-14 19:00:00,143.3400,143.55,143.28,143.4200,28764
3,2022-12-14 20:00:00,143.4400,143.44,143.25,143.4000,32010
4,2022-12-15 05:00:00,142.6800,142.73,141.20,141.4500,65218
...,...,...,...,...,...,...
95,2022-12-22 16:00:00,131.5000,132.36,131.17,132.2500,15525209
96,2022-12-22 17:00:00,132.2900,132.50,131.92,132.1726,1564856
97,2022-12-22 18:00:00,132.1867,132.23,132.05,132.1700,73548
98,2022-12-22 19:00:00,132.1900,132.19,132.00,132.0300,72303


In [5]:
'GOOGL' in script_data

True

In [6]:
'AAPL' in script_data

True

In [7]:
'NVDA' in script_data

False

In [8]:
def indicator1(df, timeperiod):
    df['indicator'] = df['close'].rolling(timeperiod).mean()
    df = df[['timestamp', 'indicator']]
    return df

In [9]:
indicator1(script_data['GOOGL'], timeperiod=5)

,timestamp,indicator
0,2022-12-14 17:00:00,NaN
1,2022-12-14 18:00:00,NaN
2,2022-12-14 19:00:00,NaN
3,2022-12-14 20:00:00,NaN
4,2022-12-15 05:00:00,94.9140
...,...,...
95,2022-12-22 16:00:00,87.1000
96,2022-12-22 17:00:00,87.1480
97,2022-12-22 18:00:00,87.3411
98,2022-12-22 19:00:00,87.5931


In [10]:
indicator1(script_data['AAPL'], timeperiod=5)

,timestamp,indicator
0,2022-12-14 17:00:00,NaN
1,2022-12-14 18:00:00,NaN
2,2022-12-14 19:00:00,NaN
3,2022-12-14 20:00:00,NaN
4,2022-12-15 05:00:00,143.00600
...,...,...
95,2022-12-22 16:00:00,131.45100
96,2022-12-22 17:00:00,131.54452
97,2022-12-22 18:00:00,131.77652
98,2022-12-22 19:00:00,132.02452


In [11]:
class Strategy(ScriptData):
    def __init__(self, script):
        self.script = script
        self.indicator_data = {}
        self.close_data = {}
        ScriptData.__init__(self)
        
    def get_script_data(self):
        ScriptData.fetch_intraday_data(self, self.script)
        data = ScriptData.convert_intraday_data(self, self.script)
        indicator_data = indicator1(data, timeperiod=5)
        self.indicator_data[self.script] = indicator_data
        data = data[['timestamp', 'close']]
        self.close_data[self.script] = data
        
    def get_signals(self):
        
        # 1. BUY (When: If indicator_data cuts close_data upwards)
        # 2. SELL (When: If indicator_data cuts close_data downwards)
        # 3. NO_SIGNAL (When: If indicator_data and close_data don’t cut
        # each other)
        
        def signal_funct(i):
            if i['indicator'] > i['close']:
                return 'BUY'
            elif i['indicator'] < i['close']:
                return 'SELL'
            else:
                return 'NO_SIGNAL'
         
        indicator_data = self.indicator_data[self.script]
        df = self.close_data[self.script]
        df = df.merge(indicator_data)
        df['indication_signal'] = df.apply(signal_funct, axis=1)
        
        ind = df.loc[:, 'indication_signal']
        timestamp = df.loc[:, 'timestamp']
        
        t = []
        d = []

        for i in range(1, len(df.index)):
            curr, prev = i, i - 1
            if ind[prev] != ind[curr]:
                t.append(timestamp[curr])
                d.append(ind[curr])

        return pd.DataFrame({
            'timestamp': t,
            'signal': d
        })

In [12]:
strategy = Strategy('NVDA')
strategy.get_script_data()
strategy.get_signals()

,timestamp,signal
0,2022-12-15 05:00:00,BUY
1,2022-12-15 15:00:00,SELL
2,2022-12-15 18:00:00,BUY
3,2022-12-16 08:00:00,SELL
4,2022-12-16 11:00:00,BUY
5,2022-12-16 15:00:00,SELL
6,2022-12-16 17:00:00,BUY
7,2022-12-19 05:00:00,SELL
8,2022-12-19 09:00:00,BUY
9,2022-12-19 13:00:00,SELL
